## **Working with Linear Regression**

## Step 1: Data Preparation

- Import the required libraries
- Load the Boston housing data set
- Prepare the data set


dataset: https://towardsdatascience.com/things-you-didnt-know-about-the-boston-housing-dataset-2e87a6f960e8

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn.datasets import fetch_openml
# https://www.openml.org/search?type=data&sort=runs&status=active

from sklearn.datasets import load_boston

ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[:

In [ ]:


# Load the Boston dataset
## boston = fetch_openml(name='boston', version=1, as_frame=True, parser='auto')
#boston = fetch_openml(name='boston', version=1, as_frame=True)

boston = load_boston()

data = boston.data
target = boston.target
feature_names = boston.feature_names

## Step 2: Create a DataFrame and Check for Missing Values

- Create a DataFrame using the Boston housing data
- Display basic statistics
- Check for missing values


In [ ]:
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['HousePrice'] = boston.target
df.head()

__Observation__:

- This is the head of the data set.

In [ ]:
df.describe()

__Observation__:
- Here, you can see the statistical analysis of the data set.

In [ ]:
df.isna().sum()

__Observation__:

- There are no empty rows in the dataset.

## Step 3: Remove Outliers from the HousePrice Column


- Use a boxplot to visualize the outliers
- Remove outliers from the __HousePrice__ column using the 1% and 99% quantiles


In [ ]:
sns.boxplot(df['HousePrice'])

__Observation__:

- There are outliers in the data set.

In [ ]:
upper_limit = df['HousePrice'].quantile(0.99)
lower_limit = df['HousePrice'].quantile(0.01)

In [ ]:
upper_limit

In [ ]:
lower_limit

In [ ]:
df['HousePrice'] = np.where(df['HousePrice'] < lower_limit, lower_limit, df['HousePrice'])
df['HousePrice'] = np.where(df['HousePrice'] > upper_limit, upper_limit, df['HousePrice'])

## Step 4: Test for Linearity of the Model

- Fit the model and display the summary
- Define functions to calculate residuals and plot actual vs. predicted values
- Test for linearity


In [ ]:
import statsmodels.api as sm
import numpy as np
import pandas as pd

boston.data = boston.data.apply(pd.to_numeric)

# X_constant = sm.add_constant(np.asarray(boston.data))
X_constant = sm.add_constant(boston.data)

# X_constant is the constant of the model (b)
# y = mX+b
# It will add 1 to each value in the dataset.

# boston_model = sm.OLS(boston.target, np.asarray(boston.data)).fit()
boston_model = sm.OLS(boston.target, X_constant).fit()
boston_model.summary()

In [ ]:
## The independent variables:
# CRIM          0
# ZN            0
# INDUS         0
# CHAS          0
# NOX           0
# RM            0
# AGE           0
# DIS           0
# RAD           0
# TAX           0
# PTRATIO       0
# B             0
# LSTAT         0

## The dependent variable:
# HousePrice    0

## The model
# y = coef+ax1+bx2+cx3+dx4+ex5+...nx13

# the weights of the model represent the importance, influence, or correlation of the independent variables


Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] 

- Define a function to calculate residual values by taking the actual and predicted values
- The value of residuals is equal to the difference between the actual and  predicted values.  

In [ ]:
def calculate_residuals(model, features, label):
    predictions =  model.predict(features)
    df_results = pd.DataFrame({'Actual' : label, 'Predicted' : predictions})
    df_results['Residuals'] = abs(df_results['Actual']) - abs(df_results['Predicted'])
    return df_results

- Next, define a function to plot the actual and predicted values using __lmplot__.
- The orange line will show the fitted line created by the linear regression model.

In [ ]:
def linear_assumptions(model, features, label):
    df_results = calculate_residuals(model, features, label)

    sns.lmplot(x='Actual', y='Predicted', data=df_results, fit_reg=False, height=7)
    line_coords = np.arange(df_results.min().min(), df_results.max().max())
    plt.plot(line_coords, line_coords, color='darkorange', linestyle='--')
    plt.title('Actual vs. Predicted')
    plt.show()

- Now, run the function __linear_assumptions__ to show the graph with the model as __boston_model__, features as __boston.data__, and label as __boston.taget__ variables as arguments.

In [ ]:
X_constant = sm.add_constant(np.asarray(boston.data))

In [ ]:
# linear_assumptions(boston_model, boston.data, boston.target)
linear_assumptions(boston_model, X_constant, boston.target)

__Observation__:

- We can observe that the line does not represent all the data points.

## Step 5: Check for Multicollinearity

- Let's check the correlation between the variables in the data set.



In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

- Calculate the variance inflation factor (VIF) for each feature
- Import the __variance_inflation_factor__ module from the __statsmodels.stats.outliers_influence__ library
- Set the features as the DataFrame, except the target variable
- Assign the __vif_data__ to the feature columns

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd
import numpy as np

x = df.drop(['HousePrice'], axis=1)
x = x.astype(float)  # Convert the array to float type

vif_data = pd.DataFrame()
vif_data['Features'] = x.columns

vif_data['vif'] = [variance_inflation_factor(x.values, i) for i in range(len(x.columns))]
print(vif_data)

__Observation__:
- From the above output, we can infer that the columns **NOX, RM, AGE,** and **PTRATIO** have higher multicollinearity. Hence, we can drop them. 

## Step 6: Remove Multicollinear Features and Split the Data set

- Remove highly multicollinear features from the data set


In [ ]:
df1 = df.drop(['NOX', 'RM', 'AGE', 'PTRATIO'], axis = 1)

- Now, set the feature and target variables.

In [ ]:
x = df1.drop(['HousePrice'], axis =1) #features
y = df1['HousePrice'] #target

- Next, split the data into train and test sets.

In [ ]:
from sklearn.model_selection import train_test_split

#Split the data
X_train, X_test, y_train, y_test  = train_test_split(x, y, random_state=0, test_size=0.25)


## Step 7: Fit the Model and Evaluate Performance

- Fit the model using OLS and display the summary.


In [ ]:
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

X_train = X_train.apply(pd.to_numeric, errors='coerce')

X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

print(X_train.dtype)
print(y_train.dtype)
print(np.isnan(X_train).sum())
print(np.isnan(y_train).sum())

model = sm.OLS(y_train, X_train).fit()

print(model.summary())

We did not included a coefficient terms so the R2 might not be as accurate as expected

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn import linear_model

- Now, fit the model using linear regression.


In [ ]:
reg = linear_model.LinearRegression()

reg.fit(X_train, y_train)

y_pred_train = reg.predict(X_train)
y_pred_test = reg.predict(X_test)

X_test = pd.DataFrame(X_test)  # Convert X_test to a pandas DataFrame
X_test = X_test.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN
X_test = np.asarray(X_test)  # Convert X_test to a numpy array

y_pred_test = reg.predict(X_test)  # Predict using the trained model

- Evaluate the model using various metrics such as the **r2_score, mean_absolute_error, and mean_squared_error**

Values for Train data:

In [ ]:
print("R Square: {}".format(r2_score(y_train, y_pred_train)))
#the higher the value the better (close to 1)

print("MAE: {}".format(mean_absolute_error(y_train, y_pred_train)))
print("MSE: {}".format(mean_squared_error(y_train, y_pred_train)))
# the lower the values the better.
# this are not so good

__Observation__:

- From the above output, we can observe that the model is a moderate fit for the given data.

Values for test data:

In [ ]:
print("R Square: {}".format(r2_score(y_test, y_pred_test)))
print("MAE: {}".format(mean_absolute_error(y_test, y_pred_test)))
print("MSE: {}".format(mean_squared_error(y_test, y_pred_test)))

__Observation__:

- The model moderately explains the testing data, as indicated by the **R** **Square** value. 